Import

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from pathlib import Path

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vulan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vulan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Preprocessing

In [2]:
def process_text(path, field="text"):

        def df_from_txt_full(path):
            content = Path(path).read_text(encoding="utf-8")
            return pd.DataFrame({"text": [content]})

        df = df_from_txt_full(path)
        
        df[field] = df[field].str.lower()

        for i in range(len(df)):
            post_temp=df._get_value(i, field)
            pattern = re.compile(r'https?://[a-zA-Z0-9./-]*/[a-zA-Z0-9?=_.]*[_0-9.a-zA-Z/-]*')    #to match url links present in the post
            post_temp= re.sub(pattern, ' ', post_temp)                                            #to replace that url link with space
            df._set_value(i, field,post_temp)
        
        for i in range(len(df)):
            post_temp=df._get_value(i, field)
            pattern = re.compile(r'[0-9]')                                    #to match numbers from 0 to 9
            post_temp= re.sub(pattern, ' ', post_temp)                        #to replace them with space
            pattern = re.compile('\W+')                                       #to match alphanumeric characters
            post_temp= re.sub(pattern, ' ', post_temp)                        #to replace them with space
            pattern = re.compile(r'[_+]')
            post_temp= re.sub(pattern, ' ', post_temp)
            df._set_value(i, field,post_temp)
        
        for i in range(len(df)):
            post_temp=df._get_value(i, field)
            pattern = re.compile('\s+')                                     #to match multiple whitespaces
            post_temp= re.sub(pattern, ' ', post_temp)                      #to replace them with single whitespace
            df._set_value(i, field, post_temp)

        remove_words = stopwords.words("english")
        for i in range(df.shape[0]):
            post_temp=df._get_value(i, field)
            post_temp=" ".join([w for w in post_temp.split(' ') if w not in remove_words])    #to remove stopwords
            df._set_value(i, field, post_temp)
        
        lemmatizer = WordNetLemmatizer()
        for i in range(df.shape[0]):
            post_temp=df._get_value(i, field)
            post_temp=" ".join([lemmatizer.lemmatize(w) for w in post_temp.split(' ')])   #to implement lemmetization i.e. to group together different forms of a word
            df._set_value(i, field, post_temp)
        
        vectorizer=TfidfVectorizer( max_features=5000,stop_words='english')
        vectorizer.fit(df[field])
        text=vectorizer.transform(df[field]).toarray()

        return text


<>:21: SyntaxWarning: invalid escape sequence '\W'
<>:29: SyntaxWarning: invalid escape sequence '\s'
<>:21: SyntaxWarning: invalid escape sequence '\W'
<>:29: SyntaxWarning: invalid escape sequence '\s'
C:\Users\vulan\AppData\Local\Temp\ipykernel_2800\3168999207.py:21: SyntaxWarning: invalid escape sequence '\W'
  pattern = re.compile('\W+')                                       #to match alphanumeric characters
C:\Users\vulan\AppData\Local\Temp\ipykernel_2800\3168999207.py:29: SyntaxWarning: invalid escape sequence '\s'
  pattern = re.compile('\s+')                                     #to match multiple whitespaces


In [3]:
import joblib

def mbti_pred(process_text):
    bundle = joblib.load("xgb_bundle.pkl")
    model = bundle["model"]
    classes = bundle["classes"] 
    pred = model.predict(process_text)

    pred = model.predict(process_text)     # encoded ints
    decoded = classes[pred] 
    return decoded

In [4]:
def pipeline_mbti_pred(path):
    data = process_text(path)
    pred = mbti_pred(data)
    return pred


In [5]:
pipeline_mbti_pred("test.txt")

array(['INFP'], dtype=object)